# Physics 256
## Random Number Generators

<img src="http://www.idquantique.com/wordpress/wp-content/uploads/Quantis-Appliance-Side-v2-Cropped.jpg" width=600px>

http://www.idquantique.com/random-number-generation/

## Last Time

- Error scaling for high dimensional quadrature
- Monte Carlo Integration

## Today

- Generation and testing of pseudorandom numbers
- Tower sampling


## Setting up the Notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
plt.style.use('notebook');
%config InlineBackend.figure_format = 'retina'
colors = ["#2078B5", "#FF7F0F", "#2CA12C", "#D72827", "#9467BE", "#8C574B",
            "#E478C2", "#808080", "#BCBE20", "#17BED0", "#AEC8E9", "#FFBC79", 
            "#98E08B", "#FF9896", "#C6B1D6", "#C59D94", "#F8B7D3", "#C8C8C8", 
           "#DCDC8E", "#9EDAE6"]

## Generation of Random Numbers

### What is a random number?

- There is really no such thing, definitely not on a deterministic classical computer
- Loose term applied to a sequence of independent numbers drawn randomly from some distribution
- Typically we select integer or real values on some finite domain

What of the simplest ways to generate uniformly distrubted random numbers on $[0,1]$ is the **Linear Congruential Generator** (LCG).

Consider the map (recursion relation) which generates integers between $0$ and $m-1$:

\begin{equation}
X_{n+1} = (a X_n + c) \mod m
\end{equation}

where $a$ is known as the multiplier, $c$ is the increment and $m$ the modulus.  Starting from an initial **seed** value $X_0$ we generate the list of numbers:

\begin{align*}
X_0 &= \text{seed} \newline
X_1 &= (a X_0 + c) \mod m \newline
X_2 &= \{a [a (X_0 + c) \mod m] + c\} \mod m \newline
\vdots &
\end{align*}

Then, a uniform number $x_n \in \mathcal{U}_{[0,1]}$ can be computed as:

\begin{equation}
x_n = \frac{X_n}{m} .
\end{equation}




<div class="span alert alert-success">
<h2>Programming challenge </h2>
Write a LCG function with $a=16807$, $c=0$, $m=2^{31}-1$ and $seed=332$ that generates a list of *pseudorandom* uniform numbers of length $N=1000$ on [0,1].
</div>

<!--
    m = 2**32-1
    a = 6
    c = 7
    U = 3
-->

In [ ]:
def lcg_rand(a,c,m,seed,N=1):
    '''A linear congruential pseudrandom number generator'''
    x = np.zeros([N])
    ###
    # INSERT CODE HERE
    ###
    return x

N = 1000
a,c,m,seed = 16807,0,2**31-1,332
x = lcg_rand(a,c,m,seed,N)

We can test for uniformity by examining a histogram

In [ ]:
# the histogram of the data
n, bins, patches = plt.hist(x, 20, normed=1, ec='w')
plt.xlabel('x')
plt.ylabel('p(x)')

## Optimal Values for the LCG

1. $c$ is relatively prime to m
2. $b=a-1$ is a multiple of $p$ for every prime number $p$ dividing $m$
3. $b$ is a multiple of 4 if $m$ is a multiple of 4

Numerical Recipes suggests:

\begin{align*}
a &= 1664525 \newline
c &= 1013904223 \newline
m &= 2^{32}
\end{align*}

We can also test the overall statistics (but not correlations) by looking at the mean and variance over the uniform probability distribution p(x) = 1$.

\begin{equation}
\langle x \rangle = \mu = \int_0^1 p(x) x dx 
= \int_0^1 x dx 
= \left. \frac{x^2}{2}\right \rvert_0^1 
= \frac{1}{2}
\end{equation}

\begin{equation}
\sigma^2 = \langle(x-\mu)^2\rangle = \int_0^1 \left(x-\frac{1}{2}\right)^2 dx =  \int_0^1 \left(x^2 - x +\frac{1}{4}\right) dx
= \frac{1}{3} - \frac{1}{2} + \frac{1}{4} = \frac{1}{12}
\end{equation}

so $\sigma = \frac{1}{\sqrt{12}} \simeq 0.2886$

In [ ]:
a,c,m,seed = 1664525,1013904223,2**32,13523
x = lcg_rand(a,c,m,seed,10000)

print(np.average(x),np.std(x))

We can also visually inspect for correlations by plotting $x_i vs. x_{i+1}$

In [ ]:
plt.figure(figsize=(5,5))
plt.plot(x[:-1],x[1:],'o', ms=3, mew=0)
plt.xlabel(r'$x_i$')
plt.ylabel(r'$x_{i+1}$')

## Tower Sampling

We can use the uniform distribution of (pseudo) random numbers in many ways, including sampling $N$ discrete events, each with their own probabilities $p_0,p_1,\ldots,p_{N-1}$.  Since something *must* happen, we know

\begin{equation}
\sum_{i=0}^{n-1} p_i = 1
\end{equation}

We can use our uniformly distributed random numbers $x\in \mathcal{U}_{[0,1]}$ to sample this discrete distribution by exploiting the fact that each event occupies a width $p_i$ in the probability interval. i.e. for a given random number $x$:

\begin{align*}
0 &\leftarrow 0 \le x < p_0 \newline
1 &\leftarrow p_0 \le x < p_0 + p_1 \newline
2 &\leftarrow p_0 + p_1 \le x < p_0 + p_1 + p_2 \newline
&\vdots \newline
N-1 &\leftarrow p_0 + \cdots + p_{N-2} \le x < 1 .
\end{align*}

Note that the relevant quantity here is the **cumulative probability**:

\begin{equation}
{P}_i = \sum_{k=0}^i p_k
\end{equation}

which for continuous distribution is:

\begin{equation}
{P}(x) = \int_{-\infty}^x p(x) dx
\end{equation}

In practice, we simply make a list of cumulative probabilites and figure out where to insert $x$.

In [ ]:
# Suppose we have 6 outcomes (eg. an unfair die) with the following propbabilites
p = [0.22181816, 0.16939565, 0.16688735, 0.06891783, 0.19622408, 0.17675693]

# generate the CDF
P = [np.sum(p[:i+1]) for i in range(len(p))]

plt.plot(P)
plt.xlabel('n')
plt.ylabel('P(x)')
plt.title('Cumulative Probability Distribution')

In [ ]:
# Generate N random numbers sampled according to the tower, searchsorted is *fast*
N = 10000
events = np.searchsorted(P,np.random.random(N))
plt.plot(p,'o', mec='None')
plt.hist(events, bins=len(p), normed=True, range=(-0.5,len(p) - 0.5), ec='w')
plt.xlabel('x')
plt.ylabel('p(x)')
plt.xlim(-0.5,5.5)
plt.title('Tower Sampling')

## Sampling Continous Distributions

We can extend the *tower sampling* concept to any continuous probability distribution. Our starting point will be the uniform distribution which satisfies:

\begin{equation}
p(x) dx = \left \{
\begin{array}[rcl]
dx & ; & 0 \le x \le 1 \\
0 & ; & \text{otherwise} 
\end{array}
\right.
\end{equation}

such that:

\begin{equation}
\int_{-\infty}^{\infty} p(x) dx = 1.
\end{equation}

Now, we want to sample some new random variables $y$ from some probability distribution $p(y)$.  This requires we identify a mapping $x\leftrightarrow y$ such that probability is conserved, i.e.

\begin{equation}
p(y) dy = p(x) dx \Rightarrow p(y) = \frac{dx}{dy}  p(x) .
\end{equation}

We can integrate both sides:

\begin{equation}
P(y) = \int_{-\infty}^y p(y') dy' = \int_{-\infty}^{y} \frac{dx}{dy'}  p(x) dy' = \int_0^y \frac{dx}{dy'} dy' = x(y)
\end{equation}

Therefore, if we can invert the CDF $P(y)$ we can get $y = P^{-1}(x)$ for a uniformly distributed $x$.

Let's see how this works for a few specific examples.


### Example 1
Generate a uniform random number $y$ on the domain $[a,b]$.

We have: 
\begin{equation}
p(y) = \left \{
\begin{array}[rcl]
{} \frac{1}{b-a} & ; & a \le y \le b \\
0 & ; & \text{otherwise} 
\end{array}
\right.
\end{equation}

So

\begin{equation}
x(y) =\int_a^y p(y') dy' = \int_a^y \frac{dy'}{b-a} = \left.\frac{y'}{b-a} \right \rvert_a^y = \frac{y-a}{b-a}
\end{equation}

so $y = (b-a)x + a$,  our well known result.

### Example 2
Consider sampling from the exponential distribution for $y\ge 0$:

\begin{equation}
p(y) = \lambda \mathrm{e}^{-\lambda y}.
\end{equation}

As before:

\begin{equation}
x(y) =\int_0^y p(y') dy' = \lambda \int_0^y \mathrm{e}^{-\lambda y'}dy' = \left.-\mathrm{e}^{-\lambda y'} \right \rvert_0^y = 1 - \mathrm{e}^{-\lambda y} .
\end{equation}

Solving for $x$:

\begin{align*}
\mathrm{e}^{-\lambda y} &= 1-x \newline
-\lambda y &= \ln (1-x) \newline
y &= -\frac{1}{\lambda} \ln (1-x) 
\end{align*}

which gives $y \in [0,\infty)$ for $x \in [0,1)$.

<br />
<div class="span alert alert-danger">
Note: we need to be careful with our uniform random number $x$ as it can formally take on the end point value $x=1$. 
</div>

In [ ]:
def p(y,λ):
    return λ*np.exp(-λ*y)

N = 10000
λ = 0.5
y = np.linspace(0,100,N)
plt.plot(y,p(y,λ),color=colors[0], label=r'$%3.1f\mathrm{e}^{-%3.1fy}$'%(λ,λ))

# sample y from a uniform x
x = np.random.random(N)
sampled_y = -(1/λ)*np.log(1-x)

plt.hist(sampled_y, bins=100, normed=True, ec='w', label='sampled');
plt.xlim(0,10);
plt.xlabel('y')
plt.ylabel('p(y)')
plt.title('Exponential Distribution')
plt.legend(loc='upper right')

## Next Time:
What happens if we can't analytically invert $P(y)$?